<a href="https://colab.research.google.com/github/HUJameson/Colab/blob/main/aillm_0105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

Mounted at /content/drive


In [ ]:
!pip install tiktoken
!pip install openai

In [ ]:
import openai
from sk_utils import read_sk
openai_sk = read_sk()
%env OPENAI_API_KEY=$openai_sk

In [7]:
import pandas as pd
import tiktoken


# import data/toutiao_cat_data.txt as a pandas dataframe
df = pd.read_csv('data/toutiao_cat_data.txt', sep='_!_', names=['id', 'code', 'category', 'title', 'keywords'])
df = df.fillna("")
df["combined"] = (
    "标题: " + df.title.str.strip() + "; 关键字: " + df.keywords.str.strip()
)

print("Lines of text before filtering: ", len(df))

encoding = tiktoken.get_encoding(embedding_encoding)
# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens]

print("Lines of text after filtering: ", len(df))

<ipython-input-7-e5af3fe1d838>:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('data/toutiao_cat_data.txt', sep='_!_', names=['id', 'code', 'category', 'title', 'keywords'])


Lines of text before filtering:  382688
Lines of text after filtering:  382688


In [ ]:
import os
from openai.embeddings_utils import get_embedding

openai.api_key = os.environ.get("OPENAI_API_KEY")

# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

# randomly sample 1k rows
df_1k = df.sample(1000, random_state=42)

df_1k["embedding"] = df_1k.combined.apply(lambda x : get_embedding(x, engine=embedding_model))
df_1k.to_csv("data/toutiao_cat_data_10k_with_embeddings.csv", index=False)

In [ ]:
import backoff

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def get_embedding_with_backoff(**kwargs):
    return get_embedding(**kwargs)

# randomly sample 10k rows
df_10k = df.sample(10000, random_state=42)

df_10k["embedding"] = df_10k.combined.apply(lambda x : get_embedding_with_backoff(text=x, engine=embedding_model))
df_10k.to_csv("data/toutiao_cat_data_10k_with_embeddings.csv", index=False)

In [ ]:
import backoff
from openai.embeddings_utils import get_embeddings

batch_size = 1000

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def get_embeddings_with_backoff(prompts, engine):
    embeddings = []
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i+batch_size]
        embeddings += get_embeddings(list_of_text=batch, engine=engine)
    return embeddings

# randomly sample 10k rows
df_all = df
# group prompts into batches of 100
prompts = df_all.combined.tolist()
prompt_batches = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

embeddings = []
for batch in prompt_batches:
    batch_embeddings = get_embeddings_with_backoff(prompts=batch, engine=embedding_model)
    embeddings += batch_embeddings

df_all["embedding"] = embeddings
df_all.to_parquet("data/toutiao_cat_data_all_with_embeddings.parquet", index=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

training_data = pd.read_parquet("data/20_newsgroup_with_embedding.parquet")
print(len(training_data))
print(training_data.head())
print(training_data["target"])

df =  training_data.sample(10000, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(
    list(df.embedding.values), df.target, test_size=0.2, random_state=42
)

clf = RandomForestClassifier(n_estimators=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

10640
                                                text  target      title  \
0  I was wondering if anyone out there could enli...       7  rec.autos   
1  \nIt depends on your priorities.  A lot of peo...       7  rec.autos   
2  an excellent automatic can be found in the sub...       7  rec.autos   
3  : Ford and his automobile.  I need information...       7  rec.autos   
4  \nYo! Watch the attributions--I didn't say tha...       7  rec.autos   

   n_tokens                                          embedding  
0       121  [-0.0391300804913044, 0.013502633199095726, -0...  
1       108  [-0.0011249205563217402, -0.00376517535187304,...  
2       476  [-0.018259447067975998, -0.008410007692873478,...  
3        86  [-0.012589422054588795, 0.006539034191519022, ...  
4       130  [-0.0006192282889969647, -0.011226896196603775...  
0         7
1         7
2         7
3         7
4         7
         ..
10635    11
10636    11
10637    11
10638    11
10639    11
Name: target, Length:

In [ ]:
from sklearn.linear_model import LogisticRegression

df =  training_data

X_train, X_test, y_train, y_test = train_test_split(
    list(df.embedding.values), df.target, test_size=0.2, random_state=42
)

clf = LogisticRegression()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

              precision    recall  f1-score   support

           0       0.55      0.63      0.59        89
           1       0.77      0.73      0.75       107
           2       0.72      0.75      0.73       114
           3       0.70      0.73      0.72       121
           4       0.76      0.66      0.71       111
           5       0.88      0.84      0.86       111
           6       0.75      0.79      0.77       132
           7       0.84      0.86      0.85       119
           8       0.75      0.81      0.78       104
           9       0.92      0.84      0.88       135
          10       0.94      0.89      0.91       117
          11       0.88      0.77      0.82       102
          12       0.73      0.73      0.73       107
          13       0.86      0.89      0.88       118
          14       0.83      0.88      0.85       104
          15       0.70      0.86      0.77       111
          16       0.73      0.81      0.77        91
          17       0.82    

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import  AdaBoostClassifier

df =  training_data
n_estimators=200

X_train, X_test, y_train, y_test = train_test_split(
    list(df.embedding.values), df.target, test_size=0.2, random_state=42
)

#dt_stump = DecisionTreeClassifier(max_depth=1,min_samples_leaf=1)
#dt_stump.fit(X_train, y_train)
dt_stump = LogisticRegression()
ada = AdaBoostClassifier(estimator=dt_stump,n_estimators=n_estimators)
ada.fit(X_train,  y_train)
preds = ada.predict(X_test)
probas = ada.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

              precision    recall  f1-score   support

           0       0.87      0.22      0.36        89
           1       0.64      0.71      0.68       107
           2       0.78      0.64      0.71       114
           3       0.63      0.65      0.64       121
           4       0.72      0.59      0.65       111
           5       0.97      0.77      0.85       111
           6       0.90      0.58      0.71       132
           7       0.98      0.67      0.80       119
           8       0.34      0.91      0.50       104
           9       0.96      0.77      0.86       135
          10       0.94      0.81      0.87       117
          11       0.91      0.69      0.78       102
          12       0.45      0.84      0.59       107
          13       0.76      0.86      0.80       118
          14       0.65      0.87      0.74       104
          15       0.54      0.87      0.67       111
          16       0.59      0.74      0.66        91
          17       0.94    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
